In [1]:
#Q&Aをもとに､データセットをつくってみる

In [1]:
import random
import numpy as np
import json
random.seed(0)
np.random.seed(0)

In [2]:
qa_path="database/output/qa0926.json"
context_path="database/output/context0926.json"


with open(qa_path,"r") as f:
    q_a_with_context_list=json.load(f)

with open(context_path,"r") as f:
    context_list=json.load(f)
len(q_a_with_context_list)

238917

In [3]:
context_id_set=list(set([c["context_id"] for c in q_a_with_context_list]))
len(context_id_set)

42394

In [4]:
q_a_with_context_list[0]

{'context_id': 4993,
 'question': 'What organisms are known to produce the enzyme thiaminase I?',
 'answer': 'Thiaminase I is known to be produced by certain bacteria including Paenibacillus thiaminolyticus, Burkholderia thailandensis, Paenibacillus apiarius, Clostridium sporogenes, and Clostridium botulinum. It has also been reported in some plants and fish, although the observations in fish need further evaluation regarding potential microbial origins.',
 'ref_id': 352}

In [7]:
from tqdm import tqdm
#記述式と選択式の割合
multiple_qa_ratio=0.6
#コンテキスト有無の割合
inc_context_ratio=0.9

question_datset=[]

for context_id in tqdm(context_id_set):
    #特定のcontextを持つ問題を集める
    part_question_list=[c for c in q_a_with_context_list if c["context_id"]==context_id]

    #物切れの回答類を削除
    for typ in ["question","answer"]:
        part_question_list=[c for c in part_question_list if len(c[typ])>10]
        part_question_list=[c for c in part_question_list if c[typ][-1]!=":"]
        for ng_words in ["Claude","http","URL","Oops!"]:
            part_question_list=[c for c in part_question_list if c[typ].find(ng_words)==-1]

    if len(part_question_list)<1:
        continue

    #ランダムに問題形式を決定
    if np.random.random()<multiple_qa_ratio:
        mode="mult"
    else:
        mode="gen"
    if len(part_question_list)<4:
        mode="gen"


    #質問をrandom select
    sampled_qa=random.sample(part_question_list,min(4,len(part_question_list)))
    target_qa_id=random.randint(0,min(3,len(sampled_qa)-1))
    target_qa=sampled_qa[target_qa_id]

    question_dict={
        "question":target_qa["question"],
        "answer":target_qa["answer"],
        "ref_id":target_qa["ref_id"],
    }
    #choices
    if mode=="mult":
        question_dict["choices"]=[qa["answer"] for qa in sampled_qa]

    #contextを使うか否か
    use_context=np.random.random()>inc_context_ratio

    #質問にtheseとかを含む場合は､contextを使う

    filt_keys=["the","these","those","this","its","et al"]

    for key in filt_keys:
        if key in target_qa["question"]:
            use_context=True
            break

    context=context_list[target_qa["context_id"]]
    if use_context:
        question_dict["context"]=context
    else:
        question_dict["context"]=""

    question_datset.append(question_dict)

100%|██████████| 42394/42394 [20:43<00:00, 34.08it/s]


In [8]:
with open("database/output/qa_dataset1017.json","w") as f:
    json.dump(question_datset,f,indent=1)

In [9]:
wo_context_data=[d for d in question_datset if d["context"]==""]
len(wo_context_data)

0

In [10]:
import random
random.seed(0)
random.shuffle(question_datset)

In [11]:
test_nums=1000

with open("database/output/qa_dataset_train_1017.json","w") as f:
    json.dump(question_datset[test_nums:],f,indent=1)

with open("database/output/qa_dataset_test_1017.json","w") as f:
    json.dump(question_datset[:test_nums],f,indent=1)

In [13]:
test_dataset=question_datset[:test_nums]
wo_context_data=[d for d in test_dataset if d["context"]==""]
len(wo_context_data)

292

In [14]:

choice_data=[d for d in test_dataset if "choices" in d.keys()]
len(choice_data)

500